## Book-Ratio Portfolio
### Import of packages and API call

You must just add your email to the header as this is required by the SEC and may want to chnage some variables


In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import time

#### Important to set these for your own ###
headers = {'User-Agent': "XXX@gmail.com"} # Email (SEC wants to know who is accessing it) 
fund_size = 10000 # Fund size for putting weights into 

# URL of the JSON data
url = "https://www.sec.gov/files/company_tickers.json"

companyTickers = requests.get(
    url,
    headers = headers
)
# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

We need to eliminate duplicates of companies from the dataframe

In [2]:
df_unique = companyData.drop_duplicates(subset=companyData.columns[2]).reset_index(drop= True)
print(df_unique)

         cik_str ticker                         title
0     0000320193   AAPL                    Apple Inc.
1     0000789019   MSFT                MICROSOFT CORP
2     0001045810   NVDA                   NVIDIA CORP
3     0001652044  GOOGL                 Alphabet Inc.
4     0001018724   AMZN                AMAZON COM INC
...          ...    ...                           ...
7963  0001941189  TANAF  Tian'an Technology Group Ltd
7964  0001976663   NXNT               Nexscient, Inc.
7965  0001947158  SRKKS    Public Shrek Royalties LLC
7966  0001951378  QLUNF              Qilun Group Inc.
7967  0002000762   GMTH                   GMTech Inc.

[7968 rows x 3 columns]


In [3]:
cik = '0000320193'
companyFacts = requests.get(
        f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
        headers=headers
        )
print(int(companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'][-1]['val']))
print(companyFacts.json()['facts']['us-gaap']['Liabilities']['units']['USD'][-1]['val'])


15204137000
264904000000


Now we can finally work with financial Data. So we create a loop that goes through each CIK to get the data we want from each company that is filling to the SEC.
But we have to make sure that we just get the data from the 10-K reports, because we want to predict annual data and not quarterly data.
We have to handle refilings and duplicates.

I want to calculate the current market capitalization:
For this we will have to get current market prices per share and additionally we will have to get the outstanding shares and the multiply with each other.

In [4]:
def get_current_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return  todays_data['Close'].iloc[0]

In [ ]:
df_unique['market cap'] = 0
df_unique['Book Value'] = 0

for i in range(len(df_unique['cik_str'])):
    try:
        cik = df_unique['cik_str'][i]
        companyFacts = requests.get(
        f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
        headers=headers
        )
        print('Processing: ' + df_unique['ticker'][i])
    
        shares_outstanding = int(companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'][-1]['val'])
        Assets = int(companyFacts.json()['facts']['us-gaap']['Assets']['units']['USD'][-1]['val'])
        Liabilities = int(companyFacts.json()['facts']['us-gaap']['Liabilities']['units']['USD'][-1]['val'])
        print(Assets)
        
        Book_Value = Assets - Liabilities
        current_price = get_current_price(df_unique['ticker'][i])
        market_cap = current_price * shares_outstanding
        
        df_unique.loc[i, 'market cap'] = market_cap
        df_unique.loc[i, 'Book Value'] = Book_Value

        #EDGAR rate 10 calls per sec https://www.sec.gov/filergroup/announcements-old/new-rate-control-limits
        if i % 10 -1 == 0 and i != 0:
            time.sleep(1) 
    except Exception as e:    
        print(e)
        continue

    

As visible (will executing the code before) there a number of different reasons why this might fail, either the data is not available from the SEC .json or for the company is no price data available on yfinance. To show how sever this situation we just have to count 0.
Afterwards this following code will drop the null rows.

In [ ]:
df_unique.isin([0]).sum(axis=0)
df_unique = df_unique.dropna(axis = 1)
df_unique.info()
# 

In [ ]:
df_unique = df_unique.loc[df_unique['market cap'] != 0]
print(df_unique.head(10))

Now that we have a dataframe with market capitalization and the bookvalue, we can calculate the ratio between the two to get the "safest" investments in the case of an bankruptcy 

(I know that this is not a "really" valuable insight but this notebooks primary purpose is to show that I can work with python in an financial environment)

In [ ]:
df_unique['sector'] = '0'
for index, row in df_unique.iterrows():
    ticker = yf.Ticker(row['ticker'])  # Access the 'ticker' for the current row
    info = ticker.info
    if info:
        sector = ticker.info.get('sector','Unknown' )  # Fetch the sector, use 'Unknown' if not found
    else:
        sector = 'Unknown'
    df_unique.at[index, 'sector'] = sector 

print(df_unique)
#for each in df_unique['sector']:
    #donwload market data, compare correlation and if above certain freshhold

## Portfolio Building and Optimization ##
Now lets build a simple portfolio that takes one stock from each sector. We could also group the stocks by sector and define subsectors by letting the stocks correrlate, but for that we would have to download a time periode (1year) of each stock which would take some time... 

*You could link that with a if above SMA30 then and lowest Ratio then buy but for simplicty sake.

Since we would want to have some sort of risk management, this jupyter notebook will implement a Risk parity approach.
(of course a 1/n would have been simpler, and there are other options e.g. Kelly criterion, Black-Littermann approach or the minimum variance or a mean variance optimization)



In [ ]:
df_unique['ratio'] = df_unique['Book Value']/df_unique['market cap']
print(df_unique)
    

In [ ]:
assets = pd.DataFrame(columns=df_unique.columns)

idx = df_unique.groupby('sector')['ratio'].idxmax()
max_ratio_assets = df_unique.loc[idx]
assets = df_unique.loc[idx]
print(assets)


Now that we have our portfolio, we can manage its risk.
As mentioned we will use the Risk parity approach for calculating the size of the assets.

In [42]:
import numpy as np

var = []  

for index, each in assets.iterrows():
    ticker = yf.Ticker(each['ticker'])  
    close = ticker.history(start="2023-01-01", end='2024-09-05', interval="1d")  
    print(close['Close'])
    if 'Close' in close.columns:  # Check if 'Close' exists in the data
        inverse_var1 = 1/np.var(close['Close'].pct_change())
        var.append(inverse_var1)
    else:
        print(f"Ticker {each['ticker']} does not have close price data.")

var = np.array(var) 
weights = var / var.sum()  
assets['weigths'] = weights

print("Weights:", weights)


Date
2023-01-03 00:00:00-05:00    56.075256
2023-01-04 00:00:00-05:00    56.075256
2023-01-05 00:00:00-05:00    56.714836
2023-01-06 00:00:00-05:00    56.795906
2023-01-09 00:00:00-05:00    57.595207
                               ...    
2024-08-28 00:00:00-04:00    62.150002
2024-08-29 00:00:00-04:00    62.634998
2024-08-30 00:00:00-04:00    62.599998
2024-09-03 00:00:00-04:00    62.599998
2024-09-04 00:00:00-04:00    59.830002
Name: Close, Length: 420, dtype: float64
Date
2023-01-03 00:00:00-05:00    34.032543
2023-01-04 00:00:00-05:00    35.149311
2023-01-05 00:00:00-05:00    35.669189
2023-01-06 00:00:00-05:00    36.497135
2023-01-09 00:00:00-05:00    37.151791
                               ...    
2024-08-28 00:00:00-04:00    29.020000
2024-08-29 00:00:00-04:00    28.680000
2024-08-30 00:00:00-04:00    28.610001
2024-09-03 00:00:00-04:00    27.940001
2024-09-04 00:00:00-04:00    28.090000
Name: Close, Length: 420, dtype: float64
Date
2023-01-03 00:00:00-05:00    31.084999
2023-0

Well but what does that mean in dollar terms?
We started with a declared variable of 10000$

Risk pariety has the advantage, that it works with variance, which takes into consideration how many datapoint we have.
So in theory at least it does not matter that a stock is only listed eg. 2 days ago. 

In [40]:
fund_size = 10000
output = assets['title'] + ' ' + (assets['weigths'] * fund_size).astype(str)
print(output)

275                       EIDP, Inc. 2965.466205149561
711        Warner Music Group Corp. 1400.6682475554633
7246               Polished.com Inc. 3.240815510961606
7260            Kalera Public Ltd Co 36.25176446069855
7134         DAYBREAK OIL & GAS, INC. 2.05373321512146
7855    American National Group Inc. 4206.523670199632
7257        MedAvail Holdings, Inc. 37.211775564898716
7903       Shapeways Holdings, Inc. 26.802012119553936
5458           Rafael Holdings, Inc. 681.0094480514706
4887                   Vacasa, Inc. 149.45248988412624
7387                        QVC INC 491.30412104028005
7191           VISION ENERGY Corp 0.015717248233071458
dtype: object


All done, we now have our portfolio with allocated assets.